<a href="https://colab.research.google.com/github/Kishan-prajapati-242/ATCTM/blob/main/notebooks/EC_demo_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
import pickle
import joblib
warnings.filterwarnings('ignore')

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("✓ All libraries imported successfully!")
print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

In [ ]:
class Config:
    # Model
    model_name = 'microsoft/deberta-v3-base'
    max_length = 256

    # Training
    batch_size = 16
    learning_rate = 2e-5
    num_epochs = 20
    warmup_steps = 500
    weight_decay = 0.01

    # Architecture
    use_auxiliary_features = True
    auxiliary_feature_dim = 128
    hidden_dim = 768
    num_attention_heads = 12
    num_transformer_layers = 2
    dropout_rate = 0.3
    label_smoothing = 0.1

    # Training strategy
    gradient_accumulation_steps = 2
    max_grad_norm = 1.0
    early_stopping_patience = 5

    # Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = Config()
print(f"Configuration loaded. Device: {config.device}")

In [ ]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/ATCTM/EVENT_CLASSIFICATION/EC-demo.csv')


In [ ]:

# Check missing values
print("Missing values:")
print(df.isnull().sum())
print("\n" + "="*50)

# Check data types
print("Data types:")
print(df.dtypes)
print("\n" + "="*50)

# Event type distribution
print("Event type distribution:")
event_counts = df['EVENT_TYPE'].value_counts()
print(event_counts)

# Visualize
plt.figure(figsize=(12, 6))
event_counts.plot(kind='bar')
plt.title('Distribution of Event Types')
plt.xlabel('Event Type')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Check unique values
print("\nUnique values in auxiliary columns:")
for col in ['SENTIMENT_VALENCE', 'EMOTION', 'SARCASM', 'TENSE', 'CERTAINTY', 'GENERATED']:
    if col in ['SENTIMENT_VALENCE', 'CERTAINTY']:
        print(f"{col}: {df[col].dtype} (range: {df[col].min():.2f} - {df[col].max():.2f})")
    else:
        unique_vals = df[col].dropna().unique()
        print(f"{col}: {len(unique_vals)} unique values - {list(unique_vals)[:10]}")


In [ ]:
class AuxiliaryFeatureExtractor:
    def __init__(self):
        self.emotion_encoder = LabelEncoder()
        self.tense_encoder = LabelEncoder()
        self.scaler = StandardScaler()

    def fit(self, df):
        # Handle missing values
        df_copy = df.copy()
        df_copy['EMOTION'] = df_copy['EMOTION'].fillna('neutral')
        df_copy['TENSE'] = df_copy['TENSE'].fillna('present')

        # Fit encoders
        self.emotion_encoder.fit(df_copy['EMOTION'])
        self.tense_encoder.fit(df_copy['TENSE'])

        # Fit scaler
        features = self._encode(df)
        self.scaler.fit(features)

        return self

    def _encode(self, df):
        features = []
        df_copy = df.copy()

        # SENTIMENT_VALENCE: numeric (0.0 to 1.0)
        features.append(df_copy['SENTIMENT_VALENCE'].fillna(0.5).values)

        # EMOTION: categorical
        df_copy['EMOTION'] = df_copy['EMOTION'].fillna('neutral')
        features.append(self.emotion_encoder.transform(df_copy['EMOTION']))

        # SARCASM: TRUE/FALSE to 1/0
        sarcasm_map = {'TRUE': 1, 'True': 1, True: 1, 'FALSE': 0, 'False': 0, False: 0}
        features.append(df_copy['SARCASM'].map(sarcasm_map).fillna(0).astype(int).values)

        # TENSE: categorical
        df_copy['TENSE'] = df_copy['TENSE'].fillna('present')
        features.append(self.tense_encoder.transform(df_copy['TENSE']))

        # CERTAINTY: numeric (0.0 to 1.0)
        features.append(df_copy['CERTAINTY'].fillna(0.5).values)

        # GENERATED: TRUE/FALSE to 1/0
        generated_map = {'TRUE': 1, 'True': 1, True: 1, 'FALSE': 0, 'False': 0, False: 0}
        features.append(df_copy['GENERATED'].map(generated_map).fillna(0).astype(int).values)

        return np.column_stack(features)

    def transform(self, df):
        features = self._encode(df)
        return self.scaler.transform(features)

print("✓ Auxiliary feature extractor defined")


In [ ]:
class EventDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, auxiliary_features=None, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.auxiliary_features = auxiliary_features
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        item = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

        if self.auxiliary_features is not None:
            item['auxiliary_features'] = torch.tensor(
                self.auxiliary_features[idx],
                dtype=torch.float
            )

        return item

print("✓ Dataset class defined")


In [ ]:
class MultiModalEventClassifier(nn.Module):
    def __init__(self, config, num_labels, auxiliary_feature_size=0):
        super().__init__()

        # Load pre-trained transformer
        self.transformer = AutoModel.from_pretrained(config.model_name)

        # Freeze embeddings
        for param in self.transformer.embeddings.parameters():
            param.requires_grad = False

        # Auxiliary feature processing
        self.use_auxiliary = config.use_auxiliary_features and auxiliary_feature_size > 0
        if self.use_auxiliary:
            self.auxiliary_encoder = nn.Sequential(
                nn.Linear(auxiliary_feature_size, config.auxiliary_feature_dim),
                nn.LayerNorm(config.auxiliary_feature_dim),
                nn.ReLU(),
                nn.Dropout(config.dropout_rate),
                nn.Linear(config.auxiliary_feature_dim, config.auxiliary_feature_dim),
                nn.LayerNorm(config.auxiliary_feature_dim),
                nn.ReLU()
            )

        # Attention for text features
        self.text_attention = nn.MultiheadAttention(
            embed_dim=self.transformer.config.hidden_size,
            num_heads=config.num_attention_heads,
            dropout=config.dropout_rate,
            batch_first=True
        )

        # Feature fusion
        fusion_input_dim = self.transformer.config.hidden_size
        if self.use_auxiliary:
            fusion_input_dim += config.auxiliary_feature_dim

        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(fusion_input_dim, config.hidden_dim),
            nn.LayerNorm(config.hidden_dim),
            nn.GELU(),
            nn.Dropout(config.dropout_rate),
            nn.Linear(config.hidden_dim, config.hidden_dim),
            nn.LayerNorm(config.hidden_dim),
            nn.GELU(),
            nn.Dropout(config.dropout_rate),
            nn.Linear(config.hidden_dim, num_labels)
        )

    def forward(self, input_ids, attention_mask, auxiliary_features=None):
        # Get transformer outputs
        outputs = self.transformer(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # Apply attention
        hidden_state = outputs.last_hidden_state
        attended, _ = self.text_attention(
            hidden_state, hidden_state, hidden_state,
            key_padding_mask=~attention_mask.bool()
        )

        # Pool
        text_features = (attended * attention_mask.unsqueeze(-1)).sum(dim=1) / attention_mask.sum(dim=1, keepdim=True)

        # Combine with auxiliary features
        if self.use_auxiliary and auxiliary_features is not None:
            aux_features = self.auxiliary_encoder(auxiliary_features)
            combined_features = torch.cat([text_features, aux_features], dim=-1)
        else:
            combined_features = text_features

        # Classify
        logits = self.classifier(combined_features)

        return logits

print("✓ Model architecture defined")


In [ ]:
def train_epoch(model, dataloader, optimizer, scheduler, criterion, device):
    model.train()
    total_loss = 0
    predictions = []
    true_labels = []

    for batch in tqdm(dataloader, desc='Training'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        auxiliary_features = None
        if 'auxiliary_features' in batch:
            auxiliary_features = batch['auxiliary_features'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            auxiliary_features=auxiliary_features
        )

        loss = criterion(outputs, labels)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        _, preds = torch.max(outputs, dim=1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    return total_loss / len(dataloader), accuracy

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc='Evaluating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            auxiliary_features = None
            if 'auxiliary_features' in batch:
                auxiliary_features = batch['auxiliary_features'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                auxiliary_features=auxiliary_features
            )

            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    return total_loss / len(dataloader), accuracy, predictions, true_labels

print("✓ Training functions defined")


In [ ]:
label_encoder = LabelEncoder()
df['event_type_encoded'] = label_encoder.fit_transform(df['EVENT_TYPE'])

print(f"Number of event types: {len(label_encoder.classes_)}")
print(f"Event types: {list(label_encoder.classes_)}")

# Extract auxiliary features
aux_extractor = AuxiliaryFeatureExtractor()
aux_extractor.fit(df)
auxiliary_features = aux_extractor.transform(df)
print(f"Auxiliary features shape: {auxiliary_features.shape}")

# Split data
X_train, X_val, y_train, y_val, aux_train, aux_val = train_test_split(
    df['TEXT'].values,
    df['event_type_encoded'].values,
    auxiliary_features,
    test_size=0.2,
    random_state=42,
    stratify=df['event_type_encoded']
)

print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

# Create datasets
train_dataset = EventDataset(X_train, y_train, tokenizer, aux_train, config.max_length)
val_dataset = EventDataset(X_val, y_val, tokenizer, aux_val, config.max_length)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, num_workers=2)

print(f"✓ DataLoaders created")
print(f"  - Train batches: {len(train_loader)}")
print(f"  - Val batches: {len(val_loader)}")


In [ ]:
num_labels = len(label_encoder.classes_)
model = MultiModalEventClassifier(config, num_labels, auxiliary_feature_size=auxiliary_features.shape[1])
model.to(config.device)

# Model info
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Optimizer and scheduler
optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
total_steps = len(train_loader) * config.num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=config.warmup_steps, num_training_steps=total_steps)
criterion = nn.CrossEntropyLoss(label_smoothing=config.label_smoothing)

# Training loop
print("\nStarting training...")
train_losses, train_accs = [], []
val_losses, val_accs = [], []
best_accuracy = 0
patience_counter = 0

for epoch in range(config.num_epochs):
    print(f"\nEpoch {epoch + 1}/{config.num_epochs}")

    # Train
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, scheduler, criterion, config.device)
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Evaluate
    val_loss, val_acc, _, _ = evaluate(model, val_loader, criterion, config.device)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_accuracy:
        best_accuracy = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= config.early_stopping_patience:
        print(f"Early stopping at epoch {epoch + 1}")
        break

print(f"\n✓ Training complete! Best accuracy: {best_accuracy:.4f}")

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train Accuracy')
plt.plot(val_accs, label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.tight_layout()
plt.show()


In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

# Final evaluation
val_loss, val_acc, predictions, true_labels = evaluate(model, val_loader, criterion, config.device)

print(f"Final Validation Accuracy: {val_acc:.4f}")
print("\nClassification Report:")
print(classification_report(true_labels, predictions, target_names=label_encoder.classes_))

# Confusion matrix
cm = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_, cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
torch.save({
    'model_state_dict': model.state_dict(),
    'label_encoder_classes': label_encoder.classes_,
    'emotion_encoder_classes': aux_extractor.emotion_encoder.classes_,
    'tense_encoder_classes': aux_extractor.tense_encoder.classes_,
    'scaler_mean': aux_extractor.scaler.mean_,
    'scaler_scale': aux_extractor.scaler.scale_,
    'config': config,
    'best_accuracy': best_accuracy
}, 'event_classifier_complete.pth')

print("✓ Model saved as 'event_classifier_complete.pth'")

In [ ]:

def predict_event(text, model, tokenizer, label_encoder, aux_extractor=None,
                  auxiliary_data=None, device='cpu'):
    model.eval()

    # Tokenize
    encoding = tokenizer(text, truncation=True, padding='max_length',
                        max_length=config.max_length, return_tensors='pt')

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Handle auxiliary features - ALWAYS provide them
    if aux_extractor is not None:
        if auxiliary_data is None:
            # Create default auxiliary data when not provided
            auxiliary_data = {
                'SENTIMENT_VALENCE': 0.5,  # neutral sentiment
                'EMOTION': 'neutral',
                'SARCASM': 'FALSE',
                'TENSE': 'present',
                'CERTAINTY': 0.5,
                'GENERATED': 'TRUE'
            }
        aux_df = pd.DataFrame([auxiliary_data])
        auxiliary_features = torch.tensor(
            aux_extractor.transform(aux_df), dtype=torch.float
        ).to(device)
    else:
        # This should not happen in normal use, but just in case
        raise ValueError("aux_extractor is required for predictions")

    # Predict
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                       auxiliary_features=auxiliary_features)
        probabilities = torch.softmax(outputs, dim=1)
        prediction = torch.argmax(outputs, dim=1)

    predicted_event = label_encoder.inverse_transform(prediction.cpu().numpy())[0]
    confidence = probabilities[0, prediction[0]].item()

    # Top 3 predictions
    top_probs, top_indices = torch.topk(probabilities[0], 3)
    top_predictions = [(label_encoder.inverse_transform([idx.item()])[0], prob.item())
                      for idx, prob in zip(top_indices, top_probs)]

    return {
        'predicted_event': predicted_event,
        'confidence': confidence,
        'top_predictions': top_predictions
    }

print("✓ Inference function defined")

In [ ]:

test_examples = [
    {
        'text': "I just got promoted to senior manager!",
        'aux': {'SENTIMENT_VALENCE': 0.9, 'EMOTION': 'joy', 'SARCASM': 'FALSE',
                'TENSE': 'past', 'CERTAINTY': 1.0, 'GENERATED': 'TRUE'}
    },
    {
        'text': "They laid me off after 10 years with the company",
        'aux': {'SENTIMENT_VALENCE': 0.1, 'EMOTION': 'sadness', 'SARCASM': 'FALSE',
                'TENSE': 'past', 'CERTAINTY': 1.0, 'GENERATED': 'TRUE'}
    },
    {
        'text': "Starting my freelance journey next month!",
        'aux': {'SENTIMENT_VALENCE': 0.8, 'EMOTION': 'hope', 'SARCASM': 'FALSE',
                'TENSE': 'future', 'CERTAINTY': 0.9, 'GENERATED': 'TRUE'}
    }
]

print("Testing predictions:")
print("="*60)

for i, example in enumerate(test_examples, 1):
    result = predict_event(example['text'], model, tokenizer, label_encoder,
                          aux_extractor, example['aux'], config.device)

    print(f"\nExample {i}: \"{example['text']}\"")
    print(f"Predicted: {result['predicted_event']} (confidence: {result['confidence']:.2%})")
    print(f"Top 3 predictions:")
    for event, prob in result['top_predictions']:
        print(f"  - {event}: {prob:.2%}")

# Test without auxiliary features
print("\n" + "="*60)
print("Testing without auxiliary features (using defaults):")

simple_text = "I can't believe they fired me today"
# Pass aux_extractor but no auxiliary_data - it will use defaults
result = predict_event(simple_text, model, tokenizer, label_encoder,
                      aux_extractor, None, config.device)
print(f"\nText: \"{simple_text}\"")
print(f"Predicted: {result['predicted_event']} ({result['confidence']:.2%})")

# More test examples
print("\nAdditional examples (with default auxiliary features):")
more_examples = [
    "Just got a raise after my annual review!",
    "I'm being terminated effective immediately",
    "Starting as a freelancer next week",
    "Received employee of the year award"
]

for text in more_examples:
    result = predict_event(text, model, tokenizer, label_encoder,
                          aux_extractor, None, config.device)
    print(f"\n\"{text}\"")
    print(f"→ {result['predicted_event']} ({result['confidence']:.2%})")

In [ ]:
# Cell 18: Test with Balanced Real-Life Examples
"""
Test the model with realistic but clearer examples
"""
balanced_examples = [
    # Promotions - clear but natural
    {
        'text': "Just got the call - I'm the new VP of Sales! Can't believe it!",
        'expected': 'got_promoted'
    },
    {
        'text': "Moving up to team lead position next month. Nervous but excited!",
        'expected': 'got_promoted'
    },

    # Getting fired - direct but realistic
    {
        'text': "HR just told me today is my last day. Still in shock.",
        'expected': 'got_fired'
    },
    {
        'text': "They terminated my contract. No warning, just done.",
        'expected': 'got_fired'
    },

    # Layoffs - clear context
    {
        'text': "Our entire division got laid off due to budget cuts. 200 people gone.",
        'expected': 'got_laid_off'
    },
    {
        'text': "Company downsizing hit us hard. Got my severance package today.",
        'expected': 'got_laid_off'
    },

    # Freelancing - obvious transition
    {
        'text': "Quit my job to start freelancing full-time. First client meeting tomorrow!",
        'expected': 'became_freelancer'
    },
    {
        'text': "Finally took the leap into freelance consulting. No more office politics!",
        'expected': 'became_freelancer'
    },

    # Starting business
    {
        'text': "Registered my LLC today! My side project is now my main business.",
        'expected': 'started_business'
    },
    {
        'text': "Left corporate to launch my startup. Scary but it's now or never.",
        'expected': 'started_business'
    },

    # Job changes
    {
        'text': "Last day at ABC Corp, starting at XYZ Inc on Monday!",
        'expected': 'changed_jobs'
    },
    {
        'text': "Accepted an offer from a competitor. Better pay and growth opportunities.",
        'expected': 'changed_jobs'
    },

    # Getting hired
    {
        'text': "I GOT THE JOB! Starting as a developer at Google next month!",
        'expected': 'got_hired'
    },
    {
        'text': "After 6 months of searching, finally got hired! Marketing manager role.",
        'expected': 'got_hired'
    },

    # Raise and pay cuts
    {
        'text': "Annual review went great - 15% raise effective immediately!",
        'expected': 'got_raise'
    },
    {
        'text': "Company announced 10% salary reduction across the board. This hurts.",
        'expected': 'got_pay_cut'
    },

    # Work issues
    {
        'text': "Got written up for being late three times this month. Need to do better.",
        'expected': 'got_late_to_work'
    },
    {
        'text': "Missed the client deadline and the boss is furious. Major mistake.",
        'expected': 'missed_deadline'
    },

    # Workplace harassment
    {
        'text': "Filed a complaint with HR about my manager's inappropriate comments.",
        'expected': 'workplace_harassment'
    },
    {
        'text': "Being bullied at work is affecting my mental health. Considering options.",
        'expected': 'workplace_harassment'
    },

    # Awards and recognition
    {
        'text': "Won employee of the quarter! Hard work really does pay off.",
        'expected': 'got_work_award'
    },
    {
        'text': "Received the innovation award at the company ceremony last night!",
        'expected': 'got_work_award'
    },

    # Retirement
    {
        'text': "After 35 years, I'm officially retiring next Friday. What a journey!",
        'expected': 'retired'
    },
    {
        'text': "Submitted my retirement papers. 6 more weeks and I'm done!",
        'expected': 'retired'
    },

    # Business failure
    {
        'text': "Had to shut down the business. Ran out of funding and customers.",
        'expected': 'business_failed'
    },
    {
        'text': "My startup didn't make it. Back to the job market I go.",
        'expected': 'business_failed'
    },

    # Sabbatical
    {
        'text': "Taking a 6-month sabbatical to travel and recharge. Company approved!",
        'expected': 'got_sabbatical'
    },
    {
        'text': "Unpaid leave starting next month. Need time to focus on family.",
        'expected': 'got_sabbatical'
    },

    # New career
    {
        'text': "Left engineering to become a teacher. Completely new career at 40!",
        'expected': 'started_new_career'
    },
    {
        'text': "From lawyer to chef - following my passion finally!",
        'expected': 'started_new_career'
    },

    # Demotion
    {
        'text': "They moved me back to analyst role. No longer managing the team.",
        'expected': 'got_demoted'
    },
    {
        'text': "Lost my senior title after the merger. Pretty disappointed.",
        'expected': 'got_demoted'
    }
]

print("Testing balanced real-life examples:")
print("="*70)

correct = 0
total = len(balanced_examples)

for i, example in enumerate(balanced_examples, 1):
    result = predict_event(example['text'], model, tokenizer, label_encoder,
                          aux_extractor, None, config.device)

    predicted = result['predicted_event']
    expected = example['expected']
    is_correct = predicted == expected
    if is_correct:
        correct += 1

    print(f"\n{i}. \"{example['text']}\"")
    print(f"   Expected: {expected}")
    print(f"   Predicted: {predicted} ({result['confidence']:.1%}) {'✓' if is_correct else '✗'}")

    if not is_correct and result['confidence'] < 0.8:
        print(f"   Top 3:")
        for event, prob in result['top_predictions']:
            print(f"     - {event}: {prob:.1%}")

print(f"\n{'='*70}")
print(f"Accuracy on balanced examples: {correct}/{total} ({correct/total*100:.1f}%)")

# Test a few edge cases
print(f"\n{'='*70}")
print("Testing edge cases:")

edge_cases = [
    "Promotion without a raise - still grateful for the opportunity though!",
    "They're letting me go but calling it 'mutual separation'",
    "Is it really freelancing if I only have one client who used to be my employer?",
    "Got demoted but keeping the same salary, so mixed feelings",
    "Won an award but it came with mandatory overtime",
]

for text in edge_cases:
    result = predict_event(text, model, tokenizer, label_encoder,
                          aux_extractor, None, config.device)
    print(f"\n\"{text}\"")
    print(f"→ {result['predicted_event']} ({result['confidence']:.1%})")

In [ ]:
# Cell 19: Enhanced Multi-Event Prediction
"""
Prediction function that returns multiple events for ambiguous cases
"""
def predict_event_multi(text, model, tokenizer, label_encoder, aux_extractor=None,
                       auxiliary_data=None, device='cpu', certainty_threshold=0.8):
    """
    Predicts potentially multiple events when certainty is below threshold.
    Returns events until cumulative certainty exceeds threshold.
    """
    model.eval()

    # Tokenize
    encoding = tokenizer(text, truncation=True, padding='max_length',
                        max_length=config.max_length, return_tensors='pt')

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Handle auxiliary features
    if aux_extractor is not None:
        if auxiliary_data is None:
            auxiliary_data = {
                'SENTIMENT_VALENCE': 0.5,
                'EMOTION': 'neutral',
                'SARCASM': 'FALSE',
                'TENSE': 'present',
                'CERTAINTY': 0.5,
                'GENERATED': 'TRUE'
            }
        aux_df = pd.DataFrame([auxiliary_data])
        auxiliary_features = torch.tensor(
            aux_extractor.transform(aux_df), dtype=torch.float
        ).to(device)
    else:
        raise ValueError("aux_extractor is required for predictions")

    # Predict
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                       auxiliary_features=auxiliary_features)
        probabilities = torch.softmax(outputs, dim=1)

    # Get all predictions sorted by probability
    probs, indices = torch.sort(probabilities[0], descending=True)

    # Convert to list of (event, certainty) tuples
    all_predictions = [(label_encoder.inverse_transform([idx.item()])[0], prob.item())
                      for idx, prob in zip(indices, probs)]

    # Determine primary event and certainty
    primary_event = all_predictions[0][0]
    primary_certainty = all_predictions[0][1]

    # If certainty is high enough, return single event
    if primary_certainty >= certainty_threshold:
        return {
            'events': [primary_event],
            'certainties': [primary_certainty],
            'combined_certainty': primary_certainty,
            'interpretation': f"Clear single event: {primary_event}"
        }

    # Otherwise, accumulate events until threshold is met
    selected_events = []
    selected_certainties = []
    cumulative_certainty = 0.0

    for event, certainty in all_predictions:
        if cumulative_certainty >= certainty_threshold:
            break
        selected_events.append(event)
        selected_certainties.append(certainty)
        cumulative_certainty += certainty * (1 - cumulative_certainty)  # Diminishing returns

        # Stop at 3 events max for clarity
        if len(selected_events) >= 3:
            break

    # Create interpretation
    if len(selected_events) == 1:
        interpretation = f"Likely {selected_events[0]} but with some uncertainty"
    elif len(selected_events) == 2:
        interpretation = f"Could be {selected_events[0]} or {selected_events[1]}"
    else:
        interpretation = f"Multiple possible events: {', '.join(selected_events[:2])} or others"

    return {
        'events': selected_events,
        'certainties': selected_certainties,
        'combined_certainty': cumulative_certainty,
        'interpretation': interpretation
    }

# Test the multi-event prediction
print("Testing multi-event predictions on ambiguous cases:")
print("="*70)

ambiguous_examples = [
    # Clear cases (should return single event)
    "I got fired yesterday. Still processing it.",
    "Promoted to Director level! Dream come true!",

    # Multi-event cases
    "Left my job for a better position at a competitor with 30% more pay",
    "They promoted me but I'm still doing my old job too with no raise",
    "Company shut down so I'm freelancing with my former clients",
    "Got the award but they also cut my hours",
    "Quit to start my own consulting firm",
    "The restructuring meant I lost my team but kept my title",
    "New job starts Monday - less money but better work-life balance",
    "They're calling it a sabbatical but we all know I'm not coming back",
    "Performance review was bad, lost my bonus and got a warning",
    "Accepted the severance package and already have interviews lined up"
]

for text in ambiguous_examples:
    result = predict_event_multi(text, model, tokenizer, label_encoder,
                                aux_extractor, None, config.device,
                                certainty_threshold=0.8)

    print(f"\n\"{text}\"")
    if len(result['events']) == 1:
        print(f"→ {result['events'][0]} (certainty: {result['certainties'][0]:.2f})")
    else:
        print(f"→ Multiple events detected:")
        for event, cert in zip(result['events'], result['certainties']):
            print(f"   - {event}: {cert:.2f}")
        print(f"   Combined certainty: {result['combined_certainty']:.2f}")
    print(f"   Interpretation: {result['interpretation']}")

# Create a summary comparison
print(f"\n{'='*70}")
print("Comparison of single vs multi-event predictions:")

comparison_texts = [
    "Accepted offer from competitor with better pay and growth",
    "Promotion without raise - still grateful though",
    "Started freelancing after the layoff"
]

for text in comparison_texts:
    # Single prediction
    single = predict_event(text, model, tokenizer, label_encoder,
                          aux_extractor, None, config.device)

    # Multi prediction
    multi = predict_event_multi(text, model, tokenizer, label_encoder,
                               aux_extractor, None, config.device, 0.8)

    print(f"\n\"{text}\"")
    print(f"Single: {single['predicted_event']} ({single['confidence']:.1%})")
    print(f"Multi:  {' + '.join(multi['events'])} (combined: {multi['combined_certainty']:.2f})")